# Installing libraries

In [ ]:
!pip install -r https://raw.githubusercontent.com/McGill-MMA-EnterpriseAnalytics/SafeRide_Dtection/refs/heads/main/requirements.txt

Ignoring colorama: markers 'python_version >= "3.11" and python_version < "4.0" and platform_system == "Windows"' don't match your environment
Ignoring pywin32: markers 'python_version >= "3.11" and python_version < "4.0" and sys_platform == "win32"' don't match your environment
Ignoring waitress: markers 'python_version >= "3.11" and python_version < "4.0" and platform_system == "Windows"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

# Adding Kaggle Authkey

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yashsethi24","key":"edbf0036ebd673985f86ae4176a42b53"}'}

In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Downloading dataset for training

In [ ]:
!kaggle datasets download -d aslanahmedov/number-plate-detection

Dataset URL: https://www.kaggle.com/datasets/aslanahmedov/number-plate-detection
License(s): copyright-authors


In [ ]:
!unzip /content/number-plate-detection.zip -d plate_data

Archive:  /content/number-plate-detection.zip
  inflating: plate_data/TEST/TEST.jpeg  
  inflating: plate_data/TEST/TEST.mp4  
  inflating: plate_data/data.yaml    
  inflating: plate_data/images/N1.jpeg  
  inflating: plate_data/images/N1.xml  
  inflating: plate_data/images/N100.jpeg  
  inflating: plate_data/images/N100.xml  
  inflating: plate_data/images/N101.jpeg  
  inflating: plate_data/images/N101.xml  
  inflating: plate_data/images/N102.jpeg  
  inflating: plate_data/images/N102.xml  
  inflating: plate_data/images/N103.jpeg  
  inflating: plate_data/images/N103.xml  
  inflating: plate_data/images/N104.jpeg  
  inflating: plate_data/images/N104.xml  
  inflating: plate_data/images/N105.jpeg  
  inflating: plate_data/images/N105.xml  
  inflating: plate_data/images/N106.jpeg  
  inflating: plate_data/images/N106.xml  
  inflating: plate_data/images/N107.jpeg  
  inflating: plate_data/images/N107.xml  
  inflating: plate_data/images/N108.jpeg  
  inflating: plate_data/images/

# Dividing dataset for training and test set

In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

# Paths
base_path = '/content/plate_data'
images_path = os.path.join(base_path, 'images')
annotations_path = os.path.join(base_path, 'images')
output_path = '/content/plate_full_split'  # New base folder with train and val

# Create output folders
for split in ['train', 'val']:
    os.makedirs(os.path.join(output_path, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(output_path, split, 'labels'), exist_ok=True)

# Step 1: Find all images that have corresponding annotations
image_files = [f for f in os.listdir(images_path) if f.endswith('.jpeg')]
annotation_files = [f.replace('.xml', '') for f in os.listdir(annotations_path) if f.endswith('.xml')]

# Keep only images that have annotations
valid_images = [f for f in image_files if f.replace('.jpeg', '') in annotation_files]

print(f"Total valid images with annotations: {len(valid_images)}")

# Step 2: Split into train and val
train_imgs, val_imgs = train_test_split(valid_images, test_size=0.2, random_state=42)  # 80% train, 20% val
splits = {'train': train_imgs, 'val': val_imgs}

# Helper: XML to YOLO TXT converter
def convert_annotation(xml_file, txt_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    with open(txt_file, 'w') as f:
        for obj in root.findall('object'):
            cls = obj.find('name').text
            # Assuming 'plate' is class 0, adjust if multiple classes later
            cls_id = 0
            xmlbox = obj.find('bndbox')
            xmin = int(xmlbox.find('xmin').text)
            xmax = int(xmlbox.find('xmax').text)
            ymin = int(xmlbox.find('ymin').text)
            ymax = int(xmlbox.find('ymax').text)

            # Convert to YOLO format
            x_center = (xmin + xmax) / 2 / width
            y_center = (ymin + ymax) / 2 / height
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height

            f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

# Step 3: Move and convert
for split_name, img_list in splits.items():
    for img_file in img_list:
        img_src = os.path.join(images_path, img_file)
        img_dst = os.path.join(output_path, split_name, 'images', img_file)
        shutil.copy(img_src, img_dst)

        # Convert annotation
        xml_file = os.path.join(annotations_path, img_file.replace('.jpeg', '.xml'))
        txt_file = os.path.join(output_path, split_name, 'labels', img_file.replace('.jpeg', '.txt'))
        convert_annotation(xml_file, txt_file)

print("✅ Full dataset with train/val split prepared successfully!")


Total valid images with annotations: 225
✅ Full dataset with train/val split prepared successfully!


# Adding Yaml file for YOLO

In [ ]:
dataset_yaml = """
path: ../plate_full
train: train/images
val: train/images

names:
  0: Plate
"""

with open('/content/plate_full/dataset.yaml', 'w') as f:
    f.write(dataset_yaml)

print("✅ dataset.yaml updated correctly with val field!")


✅ dataset.yaml updated correctly with val field!


# Training model with more epochs + Early Stopping + more training images

In [ ]:
import os
from ultralytics import YOLO

# --------------- Training Settings ----------------
model_type = 'yolov8s.pt'              # Best performing model
imgsz = 960                             # Best image size
epochs = 200                            # Increased epochs
batch_size = 16                         # You can tune if GPU allows
data_path = '/content/plate_full/dataset.yaml'  # <-- Updated dataset path (full dataset)
save_dir = '/content/yolo_runs_final'

# --------------- Start Training ----------------
run_name = f"{model_type.replace('.pt', '')}_imgsz{imgsz}_ep{epochs}_final"

# Initialize model
model = YOLO(model_type)

# Train model
results = model.train(
    data=data_path,
    epochs=epochs,
    imgsz=imgsz,
    save=True,
    project=save_dir,
    name=run_name,
    batch=batch_size,
    device=0,
    workers=4,
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    weight_decay=0.01,
    momentum=0.9,
    patience=20,               # <-- Longer patience for early stopping
    close_mosaic=20,            # Close mosaic after 20 epochs
    cache=True,
    save_period=10,             # Save checkpoints every 10 epochs
    exist_ok=True,

    # Augmentations
    augment=True,
    mosaic=0.0,
    mixup=0.0,
    hsv_h=0.01,
    hsv_s=0.3,
    hsv_v=0.2,
    flipud=0.0,
    fliplr=0.1,
    degrees=5.0,
    translate=0.05,
    scale=0.1,
    shear=0.0,
    perspective=0.005,
    amp=True,
    agnostic_nms=False,
)

print("✅ Final training completed successfully!")


Ultralytics 8.3.119 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/plate_full/dataset.yaml, epochs=200, time=None, patience=20, batch=16, imgsz=960, save=True, save_period=10, cache=True, device=0, workers=4, project=/content/yolo_runs_final, name=yolov8s_imgsz960_ep200_final, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=20, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 

100%|██████████| 755k/755k [00:00<00:00, 23.6MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

 22        [15, 18, 21]  1   2116435  ultralytics.nn.modules.head.Detect           [1, [128, 256, 512]]          
Model summary: 129 layers, 11,135,987 parameters, 11,135,971 gradients, 28.6 GFLOPs

Transferred 349/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 96.9MB/s]


...

    200/200      8.05G     0.9904     0.8565       1.46          1        960: 100%|██████████| 15/15 [00:07<00:00,  1.94it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  2.70it/s]

                   all        225        225      0.987      0.998      0.995      0.744



200 epochs completed in 0.740 hours.
Optimizer stripped from /content/yolo_runs_final/yolov8s_imgsz960_ep200_final/weights/last.pt, 22.6MB
Optimizer stripped from /content/yolo_runs_final/yolov8s_imgsz960_ep200_final/weights/best.pt, 22.6MB

Validating /content/yolo_runs_final/yolov8s_imgsz960_ep200_final/weights/best.pt...
Ultralytics 8.3.119 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:07<00:00,  1.07it/s]


                   all        225        225      0.991          1      0.995      0.703
Speed: 0.4ms preprocess, 23.1ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /content/yolo_runs_final/yolov8s_imgsz960_ep200_final
✅ Final training completed successfully!


In [ ]:
import shutil
import os

# ----------- Define paths ------------
save_dir = '/content/yolo_runs_final/yolov8s_imgsz960_ep200_final/weights'  # Path where YOLO saved the models
destination_dir = '/content/saved_model'  # Where you want to keep final files

# Create destination directory
os.makedirs(destination_dir, exist_ok=True)

# ----------- Save best.pt and last.pt ------------
shutil.copy(os.path.join(save_dir, 'best.pt'), os.path.join(destination_dir, 'best.pt'))
shutil.copy(os.path.join(save_dir, 'last.pt'), os.path.join(destination_dir, 'last.pt'))

print("✅ Model weights saved to /content/saved_model/")

# ----------- Optional: Save training plots ------------
# If you want, you can save training curves plots too
plots_dir = '/content/yolo_runs_final/yolov8s_imgsz960_ep200_final/'
plot_files = ['results.png', 'results.csv']  # Standard YOLO outputs (loss, precision-recall curves)

for plot_file in plot_files:
    plot_path = os.path.join(plots_dir, plot_file)
    if os.path.exists(plot_path):
        shutil.copy(plot_path, os.path.join(destination_dir, plot_file))

print("✅ Training plots saved (if available).")


✅ Model weights saved to /content/saved_model/
✅ Training plots saved (if available).


In [ ]:
# Zip it
!zip -r /content/saved_model.zip /content/saved_model

# Now manually download /content/saved_model.zip from the sidebar!

  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/results.csv (deflated 62%)
  adding: content/saved_model/last.pt (deflated 8%)
  adding: content/saved_model/best.pt (deflated 8%)
  adding: content/saved_model/results.png (deflated 7%)
